### Initialize Environment and Populate Blockchain Database

In [2]:
import os
import sys
from pathlib import Path

container_src_path = Path('/app/src/')
local_src_path = Path(Path.cwd(), 'src/')

# see if this src path exists.
# if it does, we are in a container.
# if not, we are in local.
if not container_src_path.exists():
    src_path = local_src_path
    in_container = False
else:
    src_path = container_src_path
    in_container = True

src_path_str = str(src_path)
if src_path_str not in sys.path:
    sys.path.insert(0, src_path_str)

from dotenv import load_dotenv

from blockchain_data_provider import (
    PersistentBlockchainAPIData,
    BlockchainAPIJSON
)

from models.base import SessionLocal
from models.bitcoin_data import Block

# see if database tables exist. if not, create them
from models import base
from sqlalchemy import inspect
from sqlalchemy.sql import text

# block_data_url = "https://blockchain.info/rawblock/{height}?format=json"
block_data_url = "http://localhost:8500/block/{height}"

DELETE_DB = False
# DELETE_DB = True

inspector = inspect(base.engine)

if DELETE_DB:
# wipe the database
    with SessionLocal() as session:
        if inspector.has_table("inputs"):
            session.execute(text('DELETE FROM inputs'))
        if inspector.has_table("outputs"):
            session.execute(text('DELETE FROM outputs'))
        if inspector.has_table("transactions"):
            session.execute(text('DELETE FROM transactions'))
        if inspector.has_table("blocks"):
            session.execute(text('DELETE FROM blocks'))
        if inspector.has_table("addresses"):
            session.execute(text('DELETE FROM addresses'))
        session.commit()

    # if Path(Path.cwd(), DATABASE_URL).exists():
    #     os.remove(Path(Path.cwd(), DATABASE_URL))

    print("Database wiped.")

if not inspector.has_table("blocks"):
    print("No data found. Database created.")

load_dotenv()


False

In [3]:
from models.base import SessionLocal
from models.bitcoin_data import Block, Tx, Input, Output, Address

def delete_block_and_related_entities(session, block_height):
    # Select IDs of inputs related to this block and delete them
    input_ids = session.query(Input.id).join(Tx, Tx.id == Input.tx_id).filter(Tx.block_height == block_height).all()
    input_ids = [id[0] for id in input_ids]
    if input_ids:
        session.query(Input).filter(Input.id.in_(input_ids)).delete(synchronize_session=False)

    # Select IDs of outputs related to this block and delete them
    output_ids = session.query(Output.id).join(Tx, Tx.id == Output.tx_id).filter(Tx.block_height == block_height).all()
    output_ids = [id[0] for id in output_ids]
    if output_ids:
        session.query(Output).filter(Output.id.in_(output_ids)).delete(synchronize_session=False)

    # Delete all transactions related to this block
    session.query(Tx).filter(Tx.block_height == block_height).delete(synchronize_session=False)

    # Finally, delete the block itself
    session.query(Block).filter_by(height=block_height).delete(synchronize_session=False)

    # Commit once after deleting all related entities and the block
    session.commit()
    print(f"Deleted block and related entities for block height {block_height}")


def delete_blocks_in_batches(session, starting_height, batch_size=100):
    while True:
        # Get a batch of block heights to delete
        block_heights = session.query(Block.height).filter(Block.height > starting_height).limit(batch_size).all()

        if not block_heights:
            break  # Exit if no more blocks to delete

        # Convert list of tuples to list of heights
        block_heights = [height[0] for height in block_heights]

        for block_height in block_heights:
            print(f"Deleting block {block_height}")
            delete_block_and_related_entities(session, block_height)


with SessionLocal() as session:
    delete_blocks_in_batches(session, 200000)

Deleting block 200002


In [2]:
from models.base import SessionLocal
from models.bitcoin_data import Block

def delete_blocks_in_batches(session, batch_size=2):
    while True:
        # Start a transaction
        session.begin()
        
        # Get a batch of block heights to delete
        block_heights = session.query(Block.height).filter(Block.height > 200000).limit(batch_size).all()

        if not block_heights:
            break  # Exit if no more blocks to delete

        # Convert list of tuples to list of heights
        block_heights = [height[0] for height in block_heights]

        # Delete blocks in the batch
        session.query(Block).filter(Block.height.in_(block_heights)).delete(synchronize_session=False)
        session.commit()
        print(f"Deleted blocks {block_heights[0]} to {block_heights[-1]}")

with SessionLocal() as session:
    delete_blocks_in_batches(session)

In [ ]:
# max_height = 100_000
max_height = 100_000

with SessionLocal() as session:
    highest_block = session.query(Block).order_by(Block.height.desc()).first()

if highest_block is not None:
    print(f"highest block: {highest_block.height}")

slow_provider = BlockchainAPIJSON(block_endpoint=block_data_url)
provider = PersistentBlockchainAPIData(data_provider=slow_provider)
with SessionLocal() as session:
    provider.populate_blocks(session, range(0,max_height+1), show_progressbar=True)

with SessionLocal() as session:
    # ensure we can get block 170
    assert len(provider.get_block(session, 170).transactions) == 2
    assert len(provider.get_block(session, 546).transactions) == 4

### Get some Data

In [2]:
from blockchain_data_provider import PersistentBlockchainAPIData

slow_provider = BlockchainAPIJSON(block_endpoint=block_data_url)
provider = PersistentBlockchainAPIData(data_provider=slow_provider)

# provider.latest_parsed_block = 2811
# with SessionLocal() as session:
#     provider.populate_block(session, 2812)

data_provider = PersistentBlockchainAPIData()
with SessionLocal() as session:
    block_170 = data_provider.get_block(session, 170)

print(f"transaction count: {len(block_170.transactions)}")

tx_12cb = block_170.transactions[1]
print(f"address of first output: {tx_12cb.outputs[0].address.addr}")
assert provider.get_output(session, 170).value == 5000000000

with SessionLocal() as session:
    block_170 = data_provider.get_block(session, 0)

assert block_170.height == 0
assert block_170.transactions[0].outputs[0].id == 0, block_170.transactions[0].outputs[0].id

tx_id = 2_519_048

with SessionLocal() as session:
    tx = data_provider.get_tx(session, tx_id)

tx.pretty_print()
print(f"Input sum: {tx.total_input_value()}")

transaction count: 2
address of first output: 1Q2TWHE3GMdB6BZKafqwxXtWAWgFt5Jvm3
Tx(id=2519048, hash=8153b1fc28117ccf84997e8acfd349d9c209698c0fa1d812f253d0becc9c46c1, index=6800288721132042)
Inputs:
	0: 1KKn (584.72)
Outputs:
	0: 1LQD (434.705)
	1: 1AMn (150.0)
Input sum: 58472000000


In [7]:
from sqlalchemy.orm import joinedload

from models.bitcoin_data import Tx, Input, Output

tx_index = 3003916113328251

with SessionLocal() as session:
    tx_obj = session.query(Tx).options(
            joinedload(Tx.inputs)
            .joinedload(Input.prev_out),
            joinedload(Tx.outputs)
            .joinedload(Output.address)    
        ).filter_by(index=tx_index).first()

for out in tx_obj.outputs:
    print(out.address.addr[:4])

1DZT
1DCb


### Count Transactions

In [ ]:
from sqlalchemy import func
from sqlalchemy.orm import joinedload

from models.bitcoin_data import Tx, Input, Output

tx_index = 3003916113328251

with SessionLocal() as session:
    tx_count = session.query(func.count(Tx.id)).scalar()

print(tx_count)
# this takes up 292MB of memory
# so 500M transactions would take up the following
mb_per_tx = 292 / tx_count
print(f"memory usage: {mb_per_tx * 500_000_000}MB")
# So, approximately 659 GB of storage space would be required for 500 million transactions.

#### Iterate over Transactions

In [3]:
from sqlalchemy import func
from sqlalchemy.orm import joinedload
from models.bitcoin_data import Tx, Input, Output
import models.base

from tqdm import tqdm
from blockchain_data_provider import PersistentBlockchainAPIData

provider = PersistentBlockchainAPIData()

max_height = 10_000

with base.SessionLocal() as session:
    tx_count = session.query(Tx)\
                      .filter(Tx.block_height <= max_height).count()

print(f"tx count: {tx_count}")
progressbar = tqdm(total=tx_count)
last_height = 0

with base.SessionLocal() as session:
    for tx in provider.get_txs_for_blocks(
        session,
        min_height=0,
        max_height=max_height,
        buffer=10_000
    ):
        string = f"{tx.block_height}: {tx.index_in_block}"
        [f"{output.address.addr}: {output.value}" for output in tx.outputs]
        assert tx.block_height >= last_height and tx.block_height-1 <= last_height, f"Error: last height: {last_height}, current height: {tx.block_height}"
        last_height = tx.block_height
        progressbar.update(1)

progressbar.close()


tx count: 10093


100%|██████████| 10093/10093 [01:25<00:00, 117.81it/s]


### Iterate Over Outputs

In [3]:
from sqlalchemy import func
from sqlalchemy.orm import joinedload
from models.bitcoin_data import Tx, Input, Output
import models.base

from tqdm import tqdm
from blockchain_data_provider import PersistentBlockchainAPIData

provider = PersistentBlockchainAPIData()

max_height = 20_000

with base.SessionLocal() as session:
    output_count = session.query(Output)\
                          .join(Tx)\
                          .filter(Tx.block_height <= max_height).count()
    
print(f"output count: {output_count}")
progressbar = tqdm(total=output_count)

with base.SessionLocal() as session:
    for output in provider.get_outputs_for_blocks(
        session,
        min_height=0,
        max_height=max_height,
        buffer=20_000
    ):
        string = f"{output.address.addr}: {output.value}"
        progressbar.update(1)

progressbar.close()




output count: 20181


100%|██████████| 20181/20181 [00:16<00:00, 1245.10it/s]


In [5]:
import csv
from pathlib import Path
import hashlib

import base58
from ecdsa import SECP256k1, SigningKey

from models.bitcoin_data import Address

file_path = Path('/', 'app', 'keys.csv')

addresses = []

def generate_keys():
    

    # Generate a private key
    private_key = SigningKey.generate(curve=SECP256k1)
    private_key_hex = private_key.to_string().hex()

    # Get the public key
    public_key = private_key.get_verifying_key().to_string("compressed").hex()

    # SHA256 hashing of the public key
    sha256 = hashlib.sha256()
    sha256.update(bytes.fromhex(public_key))
    sha256_public_key = sha256.hexdigest()

    # RIPEMD-160 hashing
    ripemd160 = hashlib.new('ripemd160')
    ripemd160.update(bytes.fromhex(sha256_public_key))
    ripemd160_public_key = ripemd160.hexdigest()

    # Add network byte (0x00 for Main Network)
    network_byte = '00'
    network_bitcoin_public_key = network_byte + ripemd160_public_key

    # Double SHA256 hashing of the network public key
    sha256 = hashlib.sha256()
    sha256.update(bytes.fromhex(network_bitcoin_public_key))
    sha256_network_public_key = sha256.digest()
    sha256 = hashlib.sha256()
    sha256.update(sha256_network_public_key)
    sha256_double = sha256.hexdigest()

    # First 4 bytes of the double SHA256 hash used as address checksum
    checksum = sha256_double[:8]

    # Adding checksum to the network public key
    full_bitcoin_public_key = network_bitcoin_public_key + checksum

    # Base58 encoding
    bitcoin_address = base58.b58encode(bytes.fromhex(full_bitcoin_public_key)).decode()
    
    return private_key_hex, public_key, bitcoin_address


address_objs = []

while len(address_objs) == 0:
    keys = {}
    for _ in range(10_000):
        private_key_hex, public_key, bitcoin_address = generate_keys()
        keys[bitcoin_address] = (private_key_hex, public_key)

    with SessionLocal() as session:
        address_objs = session.query(Address).filter(Address.addr.in_(list(keys.keys()))).all()

    if len(address_objs) == 0:
        print(".", end="")
    else:
        print(f"Found {len(address_objs)} addresses in the database.")
        # find the address that is in the database
        # find the public/private key pairs
        for address_obj in address_objs:
            public_key, private_key = keys[address_obj.addr]
            print('-------------------------------------')
            print(f"Address: {address_obj.addr}")
            print(f"Public key: {public_key}")
            print(f"Private key: {private_key}")
            print('-------------------------------------')


............................................................................................................................................................................................................

KeyboardInterrupt: 